In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import re
import sys
import time
import datetime
import json
import bz2

from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt 
%matplotlib inline

from IPython.display import display 
plt.rc('font', family='Verdana')

In [2]:

with bz2.BZ2File('./matches.jsonlines.bz2') as matches_file:
    for line in matches_file:
        match = json.loads(line)
        
        # Обработка матча
        break
print(matches_file)

In [3]:

features = pd.read_csv('features.csv', index_col='match_id')
features_test = pd.read_csv('./features_test.csv', index_col='match_id')

y = features['radiant_win'].values

display(features.head())
display(features_test.head())


,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [4]:
X = features.drop(['duration','radiant_win','tower_status_radiant',
                                    'tower_status_dire','barracks_status_radiant','barracks_status_dire'],axis=1)

X.shape

(97230, 102)

In [5]:
blank=X.count()
X=X.fillna(0).values
blank.ix[blank < np.max(blank)]

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

In [6]:
kf = KFold(n_splits=5, random_state=1, shuffle=True)

set = np.array([5,10,20,30,40,50])

for n in set:
    scores = 0
    scores = []
    
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n,max_depth=3)

    for train ,test in kf.split(X):

        clf.fit(X[train],y[train])
        predictions = clf.predict_proba(X[test])[:, 1]
        scores.append(roc_auc_score(y[test],predictions))
        score = np.mean(scores)    
       
    print(n,score)
    print ('Time elapsed:', datetime.datetime.now() - start_time)

5 0.635672292845
Time elapsed: 0:00:16.233293
10 0.664832922805
Time elapsed: 0:00:34.580520
20 0.68211403695
Time elapsed: 0:00:54.943206
30 0.689694754206
Time elapsed: 0:01:21.999284
40 0.693934191602
Time elapsed: 0:01:49.332067
50 0.697113952608
Time elapsed: 0:02:29.081594


In [7]:
for n in set:
    scores = 0
    scores = []
    
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n,max_depth=1)

    for train ,test in kf.split(X):

        clf.fit(X[train],y[train])
        predictions = clf.predict_proba(X[test])[:, 1]
        scores.append(roc_auc_score(y[test],predictions))
        score = np.mean(scores)    
       
    print(n,score)
    print ('Time elapsed:', datetime.datetime.now() - start_time)

5 0.591931271914
Time elapsed: 0:00:07.433346
10 0.62460092734
Time elapsed: 0:00:11.194587
20 0.65040007809
Time elapsed: 0:00:16.966450
30 0.666230706965
Time elapsed: 0:00:23.229249
40 0.67439260874
Time elapsed: 0:00:30.473382
50 0.68005072681
Time elapsed: 0:00:38.167633


In [8]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

set = np.array([1,2,5,10,20,50,100])

for n in set:
    scores = 0
    scores = []
    
    start_time = datetime.datetime.now()
    clf = LogisticRegression(C=n)

    for train ,test in kf.split(X):

        clf.fit(X[train],y[train])
        predictions = clf.predict_proba(X[test])[:, 1]
        scores.append(roc_auc_score(y[test],predictions))
        score = np.mean(scores)    
       
    print(n,score)
    print ('Time elapsed:', datetime.datetime.now() - start_time)

1 0.716347270307
Time elapsed: 0:00:12.180800
2 0.716347058588
Time elapsed: 0:00:11.570363
5 0.716347086149
Time elapsed: 0:00:11.222397
10 0.716347073497
Time elapsed: 0:00:11.172927
20 0.716347105321
Time elapsed: 0:00:11.639165
50 0.71634708836
Time elapsed: 0:00:12.123606
100 0.716347075646
Time elapsed: 0:00:11.696324


In [9]:

X_not_hero = features.drop(['lobby_type','r1_hero' ,'r2_hero','r3_hero','r4_hero','r5_hero',
                   'd1_hero','d2_hero','d3_hero','d4_hero','d5_hero','duration',
                   'radiant_win','tower_status_radiant', 'tower_status_dire',
                   'barracks_status_radiant','barracks_status_dire'],axis=1)

X=X_not_hero.fillna(0).values

X = scaler.fit_transform(X)

for n in set:
    scores = 0
    scores = []
    
    start_time = datetime.datetime.now()
    clf = LogisticRegression(C=n)

    for train ,test in kf.split(X):

        clf.fit(X[train],y[train])
        predictions = clf.predict_proba(X[test])[:, 1]
        scores.append(roc_auc_score(y[test],predictions))
        score = np.mean(scores)    
       
    print(n,score)
    print ('Time elapsed:', datetime.datetime.now() - start_time)

1 0.71638033903
Time elapsed: 0:00:10.128761
2 0.71638011025
Time elapsed: 0:00:10.406089
5 0.716379872983
Time elapsed: 0:00:10.603713
10 0.716379849734
Time elapsed: 0:00:10.416600
20 0.716379743839
Time elapsed: 0:00:10.341075
50 0.716379739593
Time elapsed: 0:00:10.623014
100 0.716379682383
Time elapsed: 0:00:10.467703


In [10]:
display('количество различных идентификаторов героев:', np.max(pd.unique(features['d1_hero' ])))

'количество различных идентификаторов героев:'

112

In [11]:
# N — количество различных героев в выборке
X = features.drop(['duration','radiant_win','tower_status_radiant',
                    'tower_status_dire','barracks_status_radiant','barracks_status_dire'],axis=1)


N=112
X_pick = np.zeros((X.shape[0], N))

for i, match_id in enumerate(X.index):
    for p in np.arange(5):
        X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
X_pick.shape

(97230, 112)

In [12]:


X=X_not_hero.fillna(0).values
X1=DataFrame(X).values
X2=DataFrame(X_pick).values
X=np.concatenate([X1, X2], axis=1)

X.shape


(97230, 203)

In [13]:
X = scaler.fit_transform(X)

set = np.array([1,2,5,10,20,50,100])

for n in set:
    scores = 0
    scores = []
    
    start_time = datetime.datetime.now()
    clf = LogisticRegression(C=n)

    for train ,test in kf.split(X):

        clf.fit(X[train],y[train])
        predictions = clf.predict_proba(X[test])[:, 1]
        scores.append(roc_auc_score(y[test],predictions))
        score = np.mean(scores)    
       
    print(n,score)
    print ('Time elapsed:', datetime.datetime.now() - start_time)

1 0.751856055155
Time elapsed: 0:00:18.780003
2 0.751855567991
Time elapsed: 0:00:18.769080
5 0.751855487609
Time elapsed: 0:00:18.954495
10 0.75185543254
Time elapsed: 0:00:19.564900
20 0.751855358393
Time elapsed: 0:00:20.255190
50 0.751855351995
Time elapsed: 0:00:19.045427
100 0.751855343541
Time elapsed: 0:00:19.567666
